<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SQL-Subqueries" data-toc-modified-id="SQL-Subqueries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SQL Subqueries</a></span><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#SQL-Subqueries" data-toc-modified-id="SQL-Subqueries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>SQL Subqueries</a></span></li><li><span><a href="#Subqueries-in-FROM" data-toc-modified-id="Subqueries-in-FROM-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Subqueries in <code>FROM</code></a></span></li><li><span><a href="#Note:-Subqueries-are-Like-New-Tables!" data-toc-modified-id="Note:-Subqueries-are-Like-New-Tables!-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Note: Subqueries are Like New Tables!</a></span></li><li><span><a href="#Subqueries-in-WHERE" data-toc-modified-id="Subqueries-in-WHERE-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Subqueries in <code>WHERE</code></a></span></li><li><span><a href="#Level-Up:-Common-Table-Expressions" data-toc-modified-id="Level-Up:-Common-Table-Expressions-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Level Up: Common Table Expressions</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li></ul></li><li><span><a href="#SQL-Versions" data-toc-modified-id="SQL-Versions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>SQL Versions</a></span><ul class="toc-item"><li><span><a href="#SQL-Dialects" data-toc-modified-id="SQL-Dialects-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>SQL Dialects</a></span></li><li><span><a href="#SQLite-Pros-&amp;-Cons" data-toc-modified-id="SQLite-Pros-&amp;-Cons-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>SQLite Pros &amp; Cons</a></span><ul class="toc-item"><li><span><a href="#Pros" data-toc-modified-id="Pros-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Pros</a></span></li><li><span><a href="#Cons" data-toc-modified-id="Cons-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Cons</a></span></li></ul></li><li><span><a href="#Extra-Resources:-SQL-Versions" data-toc-modified-id="Extra-Resources:-SQL-Versions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Extra Resources: SQL Versions</a></span></li></ul></li></ul></div>

# SQL Subqueries

In [1]:
import pandas as pd
import sqlite3

## Objectives

- Use SQL subqueries to nest queries
- Identify common SQL dialects and tools
- Query data from web databases

## SQL Subqueries

Like you might nest one function within another in Python, you can nest queries in SQL. We can use a **subquery** within another query to succinctly implement queries that have multiple query steps.

In [2]:
conn = sqlite3.connect('data/flights.db')

## Subqueries in `FROM`

You can use a subquery in the `FROM` clause - this is useful, for example, if you want to apply multiple aggregation functions.

Let say we want to get the average of the number of routes departing from all airports. First we'd need to get the total number of routes departing from all airports, then take the average.

In [3]:
pd.read_sql('''
SELECT 
    source AS depart_airport
    , COUNT() AS number_of_departures
FROM
    routes
GROUP BY
    source
''', conn)

depart_airport  number_of_departures
0               AAE                     9
1               AAL                    20
2               AAN                     2
3               AAQ                     3
4               AAR                     8
...             ...                   ...
3404            ZUH                    60
3405            ZUM                     2
3406            ZVK                     3
3407            ZYI                    15
3408            ZYL                     4

[3409 rows x 2 columns]

We can use this query as a subquery, and take the average of the new `number_of_departures` column.

In [4]:
pd.read_sql('''
SELECT
    AVG(number_of_departures)
FROM (
    SELECT 
        source AS depart_airport
        ,COUNT() AS number_of_departures
    FROM
        routes
    GROUP BY
        source
)
''', conn)

AVG(number_of_departures)
0                  19.848343

## Note: Subqueries are Like New Tables!

If you squint, you'll notice that the subquery is taking the place of where we might put a table!

For example, checkout the SQL we wrote in our first subquery example:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    SELECT 
        source AS depart_airport
        ,COUNT() AS number_of_departures
    FROM
        routes
    GROUP BY
        source
)
```

We could imagine that some new table that returned by the subquery existed (let's call it `airport_departures`) and be placed in place of the subquery:

```sql
SELECT 
    AVG(number_of_departures)
FROM (
    airport_departures -- Replacing subquery with this hypothetical table
) 
```

You can actually use syntax close to this with **Common Table Expressions (CTEs)** found in the [Level Up section](#Level-Up:-Common-Table-Expressions) below.

## Subqueries in `WHERE`

You can use a subquery in the `WHERE` clause - this is useful, for example, if you want to filter a query based on results from another query.

Let's say that we want to get a table with all of the departures and destinations for the flight routes, but I only want to include flights departing from the five countries with the most airports.

To do this, we'd first need to identify the five countries that have the most airports. 

In [5]:
pd.read_sql('''
SELECT 
    country 
    ,COUNT() AS number_of_airports_in_country
FROM
    airports
GROUP BY
    country
ORDER BY
    number_of_airports_in_country DESC
LIMIT 5
''', conn)

country  number_of_airports_in_country
0  United States                           1697
1         Canada                            435
2        Germany                            321
3      Australia                            263
4         Russia                            249

I could enter these results into a new query of the routes table to get the data I want.

In [6]:
pd.read_sql('''
SELECT 
    rt.source AS depart_airport
    ,rt.dest AS destination_airport
    ,ap.country AS depart_country
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    ap.country IN (
        "United States", 
        "Canada", 
        "Germany", 
        "Australia", 
        "Russia"
    )
ORDER BY 
    depart_country
''', conn)

depart_airport destination_airport depart_country
0                DRW                 SIN      Australia
1                PER                 SIN      Australia
2                MEL                 CTU      Australia
3                SYD                 CKG      Australia
4                ADL                 BNE      Australia
...              ...                 ...            ...
20330            SOW                 FMN  United States
20331            SOW                 PHX  United States
20332            SVC                 PHX  United States
20333            VIS                 LAX  United States
20334            WRL                 CYS  United States

[20335 rows x 3 columns]

This approach works but has a few limitations:

- We have to manually enter the countries to filter them
- The list of countries won't update with our data, so we'd have to monitor and manually change them in the future
- We have to look at two separate queries to understand what our code is supposed to do
- We have to run two separate queries, which might take longer than one combined query

A better solution uses a subquery to get the list of 5 countries and feed it into our WHERE clause.

In [8]:
pd.read_sql('''
SELECT 
    rt.source AS depart_airport
    ,rt.dest AS destination_airport
    ,ap.country AS depart_country
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE ap.country IN (
-- Subquery to get the 5 countries with the most airports
    SELECT 
        country 
    FROM 
        airports
    GROUP BY 
        country
    ORDER BY 
        COUNT() DESC
    LIMIT 20
)

ORDER BY
    depart_country
''', conn)

depart_airport destination_airport depart_country
0                EZE                 DFW      Argentina
1                EZE                 JFK      Argentina
2                EZE                 MIA      Argentina
3                EZE                 PUJ      Argentina
4                AEP                 MVD      Argentina
...              ...                 ...            ...
45837            SOW                 FMN  United States
45838            SOW                 PHX  United States
45839            SVC                 PHX  United States
45840            VIS                 LAX  United States
45841            WRL                 CYS  United States

[45842 rows x 3 columns]

## Level Up: Common Table Expressions

Common Table Expressions (CTEs) are a more readable way to implement subqueries, using `WITH` and `AS`.

In [9]:
pd.read_sql('''
WITH top_5_countries AS (
    SELECT 
        country 
    FROM 
        airports
    GROUP BY 
        country
    ORDER BY 
        COUNT() DESC
    LIMIT 5
) 

SELECT 
    rt.source AS depart_airport
    ,rt.dest AS destination_airport
    ,ap.country AS depart_country
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    ap.country IN top_5_countries
ORDER BY 
    depart_country
''', conn)

depart_airport destination_airport depart_country
0                DRW                 SIN      Australia
1                PER                 SIN      Australia
2                MEL                 CTU      Australia
3                SYD                 CKG      Australia
4                ADL                 BNE      Australia
...              ...                 ...            ...
20330            SOW                 FMN  United States
20331            SOW                 PHX  United States
20332            SVC                 PHX  United States
20333            VIS                 LAX  United States
20334            WRL                 CYS  United States

[20335 rows x 3 columns]

### Exercise

Create a table listing all airlines that serve the three airports with the most outbound routes.

In [11]:
pd.read_sql('''
WITH top_3_airports AS (
    SELECT 
        airports.id 
    FROM 
        airports 
        LEFT JOIN routes
            ON routes.source_id = airports.id
    GROUP BY
        airports.id
    ORDER BY 
        COUNT() DESC
    LIMIT 3
) 

SELECT DISTINCT
    rt.airline
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    rt.source_id IN top_3_airports
''', conn)

airline
0        3E
1        3M
2        3U
3        5J
4        8L
..      ...
97       WN
98       WS
99       Y4
100      Y7
101      ZH

[102 rows x 1 columns]

<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

```sql
WITH top_3_airports AS (
    SELECT 
        airports.id 
    FROM 
        airports 
        LEFT JOIN routes
            ON routes.source_id = airports.id
    GROUP BY
        airports.id
    ORDER BY 
        COUNT() DESC
    LIMIT 3
) 

SELECT DISTINCT
    rt.airline
FROM
    routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
WHERE 
    rt.source_id IN top_3_airports
```
</details>

# SQL Versions

There is no one version of SQL - there are many versions out there! What you're learning about SQL with SQLite will apply to all of them. Just keep in mind when you apply for jobs that you may see any of these listed in any given job posting, and they are all just different versions of what you know.

## SQL Dialects

As with dialects of spoken languages, SQL dialects have many commonalities but some differences in syntax and functionality.  Here are a few of the major players:

- SQLite (we've already seen this!)
- PostgreSQL (free and open-source!)
- Oracle SQL
- MySQL (half open-souce, half Oracle)
- Microsoft SQL Server
- Transact-SQL (extends MS SQL)

## SQLite Pros & Cons

We use SQLite in this course, but it has some limitations.

### Pros

- Easy to set up
- Easy to share database files
- Uses little memory

### Cons

- Limited functionality for managing users and access permissions
- Not "thread safe": two edits at the same time can mess up your data

## Extra Resources: SQL Versions

[What Is a SQL Dialect, and Which one Should You Learn?](https://learnsql.com/blog/what-sql-dialect-to-learn/)

[SQLite vs MySQL vs PostgreSQL](https://www.digitalocean.com/community/tutorials/sqlite-vs-mysql-vs-postgresql-a-comparison-of-relational-database-management-systems)

[SQL Dialect Reference](https://en.wikibooks.org/wiki/SQL_Dialects_Reference)